# TF-IDF를 활용한 랜덤포레스트 모델 구현  

입력값에 대해 TF-IDF 값으로 벡터화 진행


## 데이터 불러오기

In [1]:
import pandas as pd

DATA_IN_PATH = 'C:/Users/JS/Desktop/mbti_data/'
TRAIN_CLEAN_DATA = 'preprocessing_final2.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

train_data.head()

,Unnamed: 0,articleid,mbti,title,content,menu_id,mbti_label,e_i,n_s,f_t,j_p
0,0,277264,estj,Estj istj 차이는 뭘까요?,저는 estj인데 istj 특징을 보면 별로 저랑 다르지않은것같아요 제 주변에 es...,11,6,1,0,0,1
1,1,277263,istj,블루스 추천,동영상 n n nGrover Washington Jr Ain t No S nshin...,11,14,0,0,0,1
2,2,277216,enfp,ENFP 여자 ESTJ 남자친구랑 오래갈 수 있을까요 ㅠㅠ,안녕하세요 ENFP 여자입니다 매번 짧고 금방 식는 연애만 해 왔는데지금 남자친구가...,11,1,1,1,1,0
3,3,276899,istj,생각에 확신이 없는 ISTJ,직장밴드에 이해하기 애매한 글이 올라왔어요 제 생각에는 찬성 반대로만 물으면 안 될...,11,14,0,0,0,1
4,4,276311,estj,원하는 연애스타일 있으신가요?,저는 개인적으로 알콩달콩도 좋지만 레드벨벳 싸이코 노래같은 그런연애 해보고 싶더라구...,11,6,1,0,0,1


## 데이터 전처리

### 형태소 분석기

In [2]:
content_text = train_data['content']
content_text

0        저는 estj인데 istj 특징을 보면 별로 저랑 다르지않은것같아요 제 주변에 es...
1        동영상 n n nGrover Washington Jr Ain t No S nshin...
2        안녕하세요 ENFP 여자입니다 매번 짧고 금방 식는 연애만 해 왔는데지금 남자친구가...
3        직장밴드에 이해하기 애매한 글이 올라왔어요 제 생각에는 찬성 반대로만 물으면 안 될...
4        저는 개인적으로 알콩달콩도 좋지만 레드벨벳 싸이코 노래같은 그런연애 해보고 싶더라구...
                               ...                        
43556     요즘 제가 교회에서 하는 연애 결혼 예비학교 강의를 듣는데 자존감이 낮고 표현을 ...
43557    사실 이건 제가 불건강 infp라서 그런 거 같기도 한데말 하나하나에 너무 예민하고...
43558               면허따려고 하는데 겁도 나고 물론 케바케겠지만 안전운전 잘 되시나요 
43559    걸어가면서 생각하다가 정신팔려서 나뭇가지에 걸림 혼자서 웃긴 생각하다가 웃음 밖에 ...
43560                 그들이 사는 세계를 들여다볼테야나도 초대햐줘용 왜 내 주변엔 없 
Name: content, Length: 43561, dtype: object

In [3]:
from konlpy.tag import Okt
import re

def preprocessing(text, okt, remove_stopwords=False, stop_words=[]) :
    # 1. 한글 및 공백 제외 문자 모두 제거
    text_split = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]","", text)
    # 2. okt 객체를 사용해 형태소 단위로 나눈다
    text_split = okt.morphs(text, stem=True)
    # 3. 불용어 제거
    if remove_stopwords :
        text_split = [token for token in text_split if not token in stop_words]
        
    return text_split

In [4]:
stop_words = ['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한', '.', '..', 'ㅠ', 'ㅠㅠ', '을', '를', '에',' 하다']
okt=Okt()
clean_content = []

for content in train_data['content']:
    #비어있는 데이터에서 멈추지 않도록 문자열인 경우에만 진행
    if type(content) == str:
        clean_content.append(preprocessing(content, okt, remove_stopwords=True, stop_words=stop_words))
    else :
        clean_content.append([]) #string이 아니면 비어있는 값 추가
        
clean_content[:4]

[['저',
  'estj',
  '인데',
  'istj',
  '특징',
  '보다',
  '별로',
  '저',
  '랑',
  '다르다',
  '않다',
  '같다',
  '제',
  '주변',
  'estj',
  'istj',
  '늘다',
  '명도',
  '없다',
  '비교',
  '하다',
  '없다',
  '다른',
  '분들',
  '시기',
  '에는',
  '둘',
  '차이점',
  '보이다'],
 ['동영상',
  'n',
  'n',
  'nGrover',
  'Washington',
  'Jr',
  'Ain',
  't',
  'No',
  'S',
  'nshine',
  'n',
  'nyo',
  't',
  'e',
  'n',
  'n',
  'n',
  'nBill',
  'withers',
  '음악',
  '으로',
  '알려지다',
  '있다',
  '이건',
  'Grover',
  'Washington',
  'Jr',
  '연',
  '곡',
  '으로',
  '기다',
  '버젼',
  '이다'],
 ['안녕하다',
  'ENFP',
  '여자',
  '이다',
  '매번',
  '짧다',
  '금방',
  '식다',
  '연애',
  '만',
  '해',
  '오다',
  '지금',
  '남자친구',
  '진짜',
  '너무',
  '좋다',
  '얼굴',
  '도',
  '잘생기다',
  '다정하다',
  '가끔',
  '씩',
  '그',
  '성격',
  '적',
  '인',
  '차이',
  '느끼다',
  '직',
  '까지',
  '막',
  '크다',
  '충돌',
  '없다',
  '그리다',
  '제',
  '엄청',
  '집착',
  '하다',
  '찡찡거리다',
  '스타일',
  '인데',
  'estj',
  '분',
  '이렇다',
  '귀찮다',
  '남자친구',
  '좀',
  '버겁다',
  '같다',
  '노력',
  '하다',
  '고치다',
  '있다',
  '자다'

In [5]:
content_final=[]

for text in clean_content :
    content_mass=' '.join(text)
    content_final.append(content_mass)

content_final[:2]

['저 estj 인데 istj 특징 보다 별로 저 랑 다르다 않다 같다 제 주변 estj istj 늘다 명도 없다 비교 하다 없다 다른 분들 시기 에는 둘 차이점 보이다',
 '동영상 n n nGrover Washington Jr Ain t No S nshine n nyo t e n n n nBill withers 음악 으로 알려지다 있다 이건 Grover Washington Jr 연 곡 으로 기다 버젼 이다']

In [6]:
#내용과 라벨값 따로 리스트 지정
content = list(content_final)
label_e_i = list(train_data['e_i'])
label_n_s = list(train_data['n_s'])
label_f_t = list(train_data['f_t'])
label_j_p = list(train_data['j_p'])

content[:2]

['저 estj 인데 istj 특징 보다 별로 저 랑 다르다 않다 같다 제 주변 estj istj 늘다 명도 없다 비교 하다 없다 다른 분들 시기 에는 둘 차이점 보이다',
 '동영상 n n nGrover Washington Jr Ain t No S nshine n nyo t e n n n nBill withers 음악 으로 알려지다 있다 이건 Grover Washington Jr 연 곡 으로 기다 버젼 이다']

## TF-IDF 벡터화

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

#tf-idf 벡터화
#min_df : 설정한 값보다 특정 토큰의 df값이 더 적으면 벡터화 과정에서 제거
#analyzer : 분석하기 위한 기준 단위(word or char)
#sublinear_tf : 문서의 단어 빈도수에 대한 스무딩 여부 설정
#ngram_range : 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인가
#max_features : 각 벡터의 최대 길이, 특징의 길이 설정
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000)

vectorized_data = vectorizer.fit_transform(content)

## 학습과 검증 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split
import numpy as np

RANDOM_SEED = 42
TEST_SPLIT = 0.2

label_1 = np.array(label_e_i)
label_2 = np.array(label_n_s)
label_3 = np.array(label_f_t)
label_4 = np.array(label_j_p)

train_input_1, eval_input_1, train_label_1, eval_label_1 = train_test_split(vectorized_data, label_1, test_size=TEST_SPLIT, random_state=RANDOM_SEED)
train_input_2, eval_input_2, train_label_2, eval_label_2 = train_test_split(vectorized_data, label_2, test_size=TEST_SPLIT, random_state=RANDOM_SEED)
train_input_3, eval_input_3, train_label_3, eval_label_3 = train_test_split(vectorized_data, label_3, test_size=TEST_SPLIT, random_state=RANDOM_SEED)
train_input_4, eval_input_4, train_label_4, eval_label_4 = train_test_split(vectorized_data, label_4, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

## 모델 선언 및 학습

### Random Forest 이용
사이킷런 라이브러리의 RandomForestClassifier 객체를 사용해 구현한다.

In [9]:
from sklearn.ensemble import RandomForestClassifier

#랜덤포레스트 분류기에 100개의 의사결정 트리를 사용한다
forest_1 = RandomForestClassifier(n_estimators=1000)
forest_2 = RandomForestClassifier(n_estimators=1000)
forest_3 = RandomForestClassifier(n_estimators=1000)
forest_4 = RandomForestClassifier(n_estimators=1000)

#학습 데이터셋으로 label_1에 대한 학습 시작
forest_1.fit(train_input_1, train_label_1)
forest_2.fit(train_input_2, train_label_2)
forest_3.fit(train_input_3, train_label_3)
forest_4.fit(train_input_4, train_label_4)

RandomForestClassifier(n_estimators=1000)

## 성능 평가

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None) :
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    #roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

### E or I label

In [11]:
pred1=forest_1.predict(eval_input_1)
get_clf_eval(eval_label_1, pred1)

오차 행렬
[[5233  403]
 [2341  736]]
정확도: 0.6851, 정밀도: 0.6462, 재현율: 0.2392, F1: 0.3491


### N or S label

In [12]:
pred2=forest_2.predict(eval_input_2)
get_clf_eval(eval_label_2, pred2)

오차 행렬
[[ 408 1718]
 [ 219 6368]]
정확도: 0.7777, 정밀도: 0.7875, 재현율: 0.9668, F1: 0.8680


### F or T label

In [13]:
pred3=forest_3.predict(eval_input_3)
get_clf_eval(eval_label_3, pred3)

오차 행렬
[[2019 2037]
 [ 940 3717]]
정확도: 0.6583, 정밀도: 0.6460, 재현율: 0.7982, F1: 0.7141


### J or P label

In [14]:
pred4=forest_4.predict(eval_input_4)
get_clf_eval(eval_label_4, pred4)

오차 행렬
[[5509  425]
 [1894  885]]
정확도: 0.7338, 정밀도: 0.6756, 재현율: 0.3185, F1: 0.4329
